In [3]:
##################################### IMPORT ######################################
# package listing                                                                 #
###################################################################################
from bs4 import BeautifulSoup 
from datetime import datetime
from datetime import timedelta
import csv as csv
import requests
import pandas as pd
import time
import numpy as np
from sklearn.cluster import KMeans
import mechanicalsoup
pd.options.mode.chained_assignment = None

In [5]:
################################ MECHANICAL SOUP ##################################
# automation of buy & sell orders will need several pieces to fully automate:     #
#     1. take target cards and build URLs to feed the MS algorithm                #
#     2. observe price, place competitive buy order                               #
#     3. refresh page (at certain repeating rate) to observe following states:    #
#          a. buy order successful: prompts sell order                            #
#               i. proceed to step 4.                                             #
#          b. price change (buy order unsuccessful): prompts cancel order,        #
#               i. return to step 2.                                              #
#     4. observe price, place competitive sell order                              #
#     5. repeat of step 3. to observe status of sale, observe following states:   #
#          a. sell order successful: termination of algorithm?                    #
#          b. price change (sell order unsuccessful): prompts cancel order,       #
#               i. 4 to step 2.                                                   #
###################################################################################

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen 

#would be cool if could use session ids, return to this...
#driver = webdriver.Remote(command_executor=url,desired_capabilities={})
#driver.session_id = session_id

url = 'https://mlb19.theshownation.com/community_market/orders/completed'

driver = webdriver.Firefox()

## until i figure out a better fix, submit creds to PSN manually here, 
## then continue to next notebook section for automation 
driver.get(url)
#link = driver.find_element_by_link_text('here')
#link.click()

session_url = driver.command_executor._url 
session_id = driver.session_id

time.sleep(30) # login and wait

driver.get(url)
src=driver.page_source
soup = BeautifulSoup(src)

fnd = soup.find("div", {"class": "pagination"})
found = str(str(str(fnd).split(">")).split("<"))

# pagination
max_index = found[-75:-73]

In [6]:
completed_orders = None
name = []
price = []
date = []
dt = []
tran_type = []

# get the data
# get max page index
for i in range(1,int(max_index)+1):
    url2='https://mlb19.theshownation.com/community_market/orders/completed?page=' + str(i) + '&'

    driver.get(url2)
    src=driver.page_source
    soup = BeautifulSoup(src)

    for j in range(0,int(len(soup.find_all("td"))/3)):

        ## NAME
        name.append(soup.find_all("td")[3*j].get_text().strip())

        ## PRICE 
        if soup.find_all("td")[3*j+1].get_text()[1] == "S":
            # transaction type 
            tran_type.append(soup.find_all("td")[3*j+1].get_text().strip()[0:5])
            # price
            price.append(int(soup.find_all("td")[3*j+1].get_text().replace(",","").split("\n")[3])*0.9)
        else:
            # transaction type 
            tran_type.append(soup.find_all("td")[3*j+1].get_text().strip()[0:7])
            # price
            price.append(int(soup.find_all("td")[3*j+1].get_text().replace(",","").split("\n")[3])*(-1))

        ## DATE
        # date
        date.append(soup.find_all("td")[3*j+2].get_text().split(" ")[0])
        # datetime
        dt.append(datetime.strptime(soup.find_all("td")[3*j+2].get_text().replace(' PDT',''), '%m/%d/%Y %I:%M%p') + timedelta(hours=2))
    
completed_orders = pd.DataFrame(zip(name,tran_type,price,date,dt),columns=['Name','Tran_type','Price','Date','Datetime'])

if name:
    name.clear(), tran_type.clear(), price.clear(), date.clear(), dt.clear()

completed_orders.to_csv("completed.csv")

In [164]:
#completed_orders.to_csv("completed")
#completed_orders["Date"][0].split(" ")
#soup.find_all("td")[3*j+2].get_text().split(" ")[0]
int(soup.find_all("td")[1].get_text().split("\n")[3])

538